In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" ### Determine the GPU to be used. without commenting out this line of code, only nocuda = 9 can be selected later.
import numpy as np
import matplotlib.pyplot as plt
import csaps
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import torch             
import torch.nn as nn    
import torch.nn.functional as F    
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from PIL import Image 
import imageio 
from early_stopping import EarlyStopping
import os, gc
import random 
import pynvml
import multiprocessing
import itertools
import subprocess
import gc
from functools import partial
from pandarallel import pandarallel
import time
import shutil
import pandas as pd

In [2]:
torch.manual_seed(123) 
np.random.seed(123) 
random.seed(123) 
torch.cuda.manual_seed_all(123) 

data = pd.read_csv('./data/data_eps1.csv')
data['time'] = pd.to_datetime(data['time'])

var_x = ['so2', 'no2', 'o3', 'co', 'WSPM', 'humi', 'dewp', 'pres']
var_y = ['pm25']
var_all = ['id_eps', 'hour_eps'] + var_y + var_x

data_used = data[data['hour_eps'] <= 48][var_all].copy().dropna(axis=0, how='any')

data_used['so2'] = ((data_used['so2']-data_used['so2'].min())/(data_used['so2'].max()-data_used['so2'].min()) - 0.5)*2
data_used['no2'] = ((data_used['no2']-data_used['no2'].min())/(data_used['no2'].max()-data_used['no2'].min()) - 0.5)*2
data_used['o3'] = ((data_used['o3']-data_used['o3'].min())/(data_used['o3'].max()-data_used['o3'].min())- 0.5)*2
data_used['co'] = ((data_used['co']-data_used['co'].min())/(data_used['co'].max()-data_used['co'].min())- 0.5)*2
data_used['WSPM'] = ((data_used['WSPM']-data_used['WSPM'].min())/(data_used['WSPM'].max()-data_used['WSPM'].min())- 0.5)*2
data_used['humi'] = ((data_used['humi']-data_used['humi'].min())/(data_used['humi'].max()-data_used['humi'].min())- 0.5)*2
data_used['dewp'] = ((data_used['dewp']-data_used['dewp'].min())/(data_used['dewp'].max()-data_used['dewp'].min())- 0.5)*2
data_used['pres'] = ((data_used['pres']-data_used['pres'].min())/(data_used['pres'].max()-data_used['pres'].min())- 0.5)*2
# data_used['temp'] = ((data_used['temp']-data_used['temp'].min())/(data_used['temp'].max()-data_used['temp'].min())- 0.5)*2
data_used['pm25'] = np.log(data_used['pm25'])

n_total = 94
randind = list(range(1,n_total+1))
random.shuffle(randind)
train_ind = [randind[i] for i in range(0, math.ceil(n_total * 0.64 ))]
test_ind = [randind[i] for i in range(math.ceil(n_total * 0.64 ), math.ceil(n_total * 0.84))]
valid_ind = [randind[i] for i in range(math.ceil(n_total * 0.84), n_total)]

x_train = np.array(data_used[data_used['id_eps'].isin(train_ind)][var_x].copy())
y_train = np.array(data_used[data_used['id_eps'].isin(train_ind)][var_y].copy())
x_valid = np.array(data_used[data_used['id_eps'].isin(valid_ind)][var_x].copy())
y_valid = np.array(data_used[data_used['id_eps'].isin(valid_ind)][var_y].copy())
x_test = np.array(data_used[data_used['id_eps'].isin(test_ind)][var_x].copy())
y_test = np.array(data_used[data_used['id_eps'].isin(test_ind)][var_y].copy())

In [3]:
x_train_df = pd.DataFrame(x_train, columns=var_x)
y_train_df = pd.DataFrame(y_train, columns=var_y)
x_valid_df = pd.DataFrame(x_valid, columns=var_x)
y_valid_df = pd.DataFrame(y_valid, columns=var_y)
x_test_df = pd.DataFrame(x_test, columns=var_x)
y_test_df = pd.DataFrame(y_test, columns=var_y)

x_train_df.to_csv('./data/x_train.csv', index=False)
y_train_df.to_csv('./data/y_train.csv', index=False)

x_valid_df.to_csv('./data/x_valid.csv', index=False)
y_valid_df.to_csv('./data/y_valid.csv', index=False)

x_test_df.to_csv('./data/x_test.csv', index=False)
y_test_df.to_csv('./data/y_test.csv', index=False)